# Ceilometer post processing for IPFS

In [ ]:
import xarray as xr
import numpy as np
import glob
import matplotlib.pyplot as plt
from matplotlib.ticker import AutoMinorLocator, AutoLocator
import yaml


In [ ]:
def add_gl_attrs(data, global_attrs):
    for key, value in global_attrs.items():
        if key in data.attrs:
            # Fallunterscheidung je nach Typ des bestehenden Attributs
            if isinstance(data.attrs[key], str):
                data.attrs[key] = f"{value}\n{data.attrs[key]}"
            elif isinstance(data.attrs[key], list):
                if isinstance(value, list):
                    data.attrs[key] = [value, data.attrs[key]]
            else:
                # Für andere Typen: in Liste umwandeln und beide anhängen
                data.attrs[key] = [value, data.attrs[key]]
        else:
            data.attrs[key] = value
    return data

## Prepare data

read config yaml which contains the data path and global attributes to be added to the dataset

In [ ]:
with open("ceilometer_config.yaml", "r") as f:
    config = yaml.safe_load(f)

In [ ]:
global_attrs = config['global_attrs']
#global_attrs

In [ ]:
data_path = config['paths']['data_path']
data_path_out = config['paths']['data_path_out']

In [ ]:
file_list = glob.glob(data_path + '*_FSMETEOR_CHM170158.nc')
file_list.sort()

In [ ]:
#file_list

## Write dataset into zarr

In [ ]:
for file in file_list:
    #read hatpro file
    data = xr.open_dataset(file)

    # Add global attributes according to IPFS
    data = add_gl_attrs(data, global_attrs)

    filename = file.split('/')[-1].split('.')[0]

    data.to_zarr(data_path_out + filename + ".zarr", mode="w")

    data.close()